# woah

In [2]:
import pandas as pd
data = pd.read_csv("archive/S01G1AllChannels.csv")
data.head(4)

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,Unnamed: 14
0,-33.0205,-15.1846,-42.1795,1.6872,42.1793,-1.68720,-5.5436,-3.6154,25.7899,-9.8819,5.5436,7.47180,11.8101,17.1128,NaN
1,-28.6291,-20.0583,-42.5410,-10.4653,35.3100,-15.68600,-19.3110,-2.4344,17.4933,3.2442,18.7081,5.09510,17.3683,3.0708,NaN
2,-21.8497,-10.9006,-32.0346,-2.3656,39.6993,-0.64483,-4.0523,-1.0830,26.8081,-3.4584,8.1861,8.40480,15.1209,9.3940,NaN
3,-25.1185,-10.9702,-32.7641,-3.4287,32.7378,4.69650,-8.6299,-1.7412,16.7637,-9.7586,1.1868,0.91086,4.3315,8.1073,NaN
